# C1 W3 Group 8 - Task 4

In [ ]:
from tqdm import tqdm
from PIL import Image

from src.background import get_painting_masks
from src.denoising import denoise_image
from src.paths import QSD2_W3_PATH, WEEK_3_RESULTS_PATH
from src.utils import show_images

In [ ]:
query_d2_image_PIL_list = [Image.open(query_img_path) for query_img_path in sorted(QSD2_W3_PATH.glob("*.jpg"))]  # Load once
query_d2_mask_PIL_list = [Image.open(query_img_path) for query_img_path in sorted(QSD2_W3_PATH.glob("*.png"))]  # Load once

In [ ]:
# Apply median filter
query_d2_image_PIL_list = [denoise_image(image) for image in query_d2_image_PIL_list]

In [ ]:
# Parameters
mask_output_folder = WEEK_3_RESULTS_PATH / "Task_3" / "final_results"
mask_output_folder.mkdir(parents=True, exist_ok=True)

painting_masks_list = []
pbar = tqdm(zip(sorted(QSD2_W3_PATH.glob("*.jpg")), query_d2_image_PIL_list, query_d2_mask_PIL_list))
for name, image, gt_mask in pbar:
    pbar.set_description(f"Processing {name.stem}")
    painting_masks = get_painting_masks(image)    
    painting_masks_list.append(painting_masks)

In [ ]:
pbar = tqdm(zip(sorted(QSD2_W3_PATH.glob("*.jpg")), query_d2_image_PIL_list, query_d2_mask_PIL_list, painting_masks_list))

for name, image, gt_mask, predicted_mask in pbar:
    pbar.set_description(f"Showing {name.stem}")
    show_images([image, gt_mask, predicted_mask])